<a href="https://colab.research.google.com/github/nikhithatemburu/IIIT-FMML-ASSIGNMENT/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**1.How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?**
The size of the validation set can have a significant impact on the accuracy estimation and the model's performance. Here's how it can be affected:

Increasing the Percentage of the Validation Set:

Increased Robustness: With a larger validation set, you're evaluating the model on a more representative portion of the data. This can lead to a more reliable estimate of how well the model will generalize to new, unseen data.

Better Detection of Overfitting: A larger validation set can make it easier to detect overfitting. If a model is overfitting, it will have a harder time performing well on a larger and more representative validation set.

Reduced Variance in Estimates: With a larger validation set, the estimate of the model's performance is likely to have lower variance. This means that the performance estimate is less sensitive to the particularities of the validation split.

Reducing the Percentage of the Validation Set:

Increased Variability in Estimates: With a smaller validation set, the estimate of the model's performance can be more variable. It becomes more dependent on the specific data points in the validation set, which may not be fully representative of the overall distribution.

Potential Overfitting to Validation Data: If the validation set is very small, the model may start to learn specific features of the validation set rather than generalizing to the broader data distribution. This can lead to overfitting to the validation data.

Less Robustness to Data Variability: A smaller validation set is more sensitive to variations in the data. If the validation set happens to have particularly challenging examples, it can give a misleading estimate of the model's true generalization performance.

In summary, increasing the percentage of the validation set generally leads to more robust and reliable estimates of model performance, while reducing it can lead to more variable and potentially less reliable estimates. However, it's important to strike a balance based on the available data and the computational resources at hand. A good rule of thumb is to aim for a validation set that's large enough to provide a meaningful evaluation but not so large that it significantly impacts the size of the training set.


**2.How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?**
The sizes of the training and validation sets play a crucial role in how well the performance on the validation set can predict the performance on the test set. Here's how they affect this prediction:

Balanced Sizes:

When the sizes of the training and validation sets are balanced and representative of the overall dataset, the performance on the validation set tends to provide a good estimate of how well the model will perform on the test set.
Small Validation Set Relative to Training Set:

If the validation set is much smaller than the training set, the estimate of model performance on the validation set may be less reliable. This is because the validation set might not adequately capture the variability in the data.
Small Training Set Relative to Validation Set:

If the training set is much smaller than the validation set, the model may not have enough data to learn the underlying patterns, which can lead to poor generalization even if it performs well on the validation set.
Large Validation Set Relative to Training Set:

A large validation set relative to the training set can potentially lead to overfitting on the validation set. The model might start to learn specific features of the validation set rather than generalizing to the broader data distribution.
Large Training Set Relative to Validation Set:

If the training set is much larger than the validation set, the model may perform well on the validation set but fail to generalize to the test set, particularly if the validation set doesn't adequately represent the variability in the data.
In summary, having both a reasonably sized training set and a representative validation set is crucial for making accurate predictions about how well the model will perform on the test set. A balanced split of the data helps ensure that the validation set provides a reliable estimate of the model's true generalization performance. It's important to avoid extremes where one set is significantly larger or smaller than the other, as this can lead to biases in the performance estimates.



**3.What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?**

There's no one-size-fits-all answer to the question of what percentage to reserve for the validation set, as the ideal split depends on various factors, including the size of your dataset, the complexity of your model, and the nature of your data.

However, a common practice is to use a 70-30 or 80-20 split for training and validation sets, respectively. That is, you allocate 70% or 80% of your data for training and the remaining 30% or 20% for validation.

Here's a breakdown of the considerations for each case:

70-30 Split:

Advantages:
More data for training can help the model learn better representations and patterns.
A relatively large validation set can provide a good estimate of model performance.
Considerations:
With a smaller validation set, there's a risk of overfitting to the validation data if the model is too complex.
80-20 Split:

Advantages:
A larger validation set provides a more stable estimate of model performance.
With a slightly smaller training set, there may be less risk of overfitting.
Considerations:
The training set is smaller, so the model might not learn as robustly.
Remember, these are general guidelines and the best split may vary based on the specifics of your dataset and problem. Additionally, if your dataset is very small, you might need to allocate a larger portion for training.

It's also worth considering techniques like cross-validation, where you repeatedly split the data into different training and validation sets, to get a more robust estimate of model performance. This can be especially useful when the dataset is small or when you want to make sure the performance estimate is not overly sensitive to a particular split.




## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**1.Does averaging the validation accuracy across multiple splits give more consistent results?**
Reduced Variance: Averaging over multiple splits helps reduce the impact of randomness or variability in the data. If a single split happens to be particularly unrepresentative, it won't dominate the evaluation.

Better Generalization: By evaluating the model on multiple different subsets of the data, you get a better estimate of how well it will perform on unseen data in general. This can be especially important when the dataset is small or has some inherent variability.

Robustness to Dataset Imbalances: If the data is imbalanced (i.e., one class has significantly more samples than another), different splits may highlight different challenges. Averaging over these splits gives a more comprehensive view of the model's performance.

Identifying Overfitting: It can help identify if the model is overfitting. If the model performs significantly better on the training data compared to the validation data across all splits, it's a sign of overfitting.

Better Confidence Intervals: When you have multiple validation scores, you can also calculate confidence intervals or statistical tests to quantify the uncertainty around the estimated performance.

Parameter Tuning: When doing hyperparameter tuning, cross-validation can be used to assess how different sets of hyperparameters affect model performance in a more robust way.

However, it's worth noting that cross-validation can be computationally expensive, especially if the model is complex or the dataset is very large. In some cases, a single train-test split may be sufficient, but cross-validation is generally recommended when it's feasible.

The choice of how many splits (e.g., 5-fold, 10-fold, etc.) to use in cross-validation depends on the specific dataset, but 5 or 10-fold cross-validation are common choices. Keep in mind that as the number of folds increases, the computational cost also increases.






**2.Does it give more accurate estimate of test accuracy?**
Cross-validation, while providing a more reliable estimate of a model's performance compared to a single train-test split, doesn't directly give a more accurate estimate of the test accuracy.

In a typical machine learning workflow, you use cross-validation on your training data to evaluate the model's performance during development and hyperparameter tuning. The test accuracy is then assessed on a separate, held-out test set that the model has never seen before.

The purpose of this separate test set is to give you an unbiased estimate of how well the model is likely to perform on completely new, unseen data. This is crucial for understanding how the model will generalize to real-world, out-of-sample data.

Using cross-validation on the training set doesn't replace the need for a separate test set, but it helps you make more informed decisions about the model's performance during development. It provides insights into how well the model is learning patterns from the data, which hyperparameters are working best, and if the model is overfitting.

So, in summary, while cross-validation aids in model development and hyperparameter tuning, it doesn't directly replace or enhance the estimate of test accuracy. The held-out test set remains a critical part of the evaluation process.







**3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**
More Stable Estimate: As the number of iterations increases, the estimate of the model's performance tends to become more stable and less sensitive to the particularities of any one random split. This means that with a larger number of iterations, you are likely to get a more reliable estimate of the model's true performance.

Increased Computational Cost: However, more iterations come at a computational cost. Each additional fold requires training and evaluating the model multiple times, which can be time-consuming, especially for complex models or large datasets.

Reduced Bias, Potentially Increased Variance: With a larger number of iterations, the bias of the estimate tends to decrease. This is because you're using more of the data for both training and validation. However, there's a trade-off, as increasing the number of folds may also lead to an increase in variance. This is because with more folds, each training set becomes smaller, potentially making it harder for the model to learn the underlying patterns.

Diminishing Returns: There are diminishing returns associated with increasing the number of iterations. Going from, say, 3-fold to 5-fold cross-validation can make a significant difference. However, going from 10-fold to 15-fold might not provide as much additional benefit.

Practical Considerations: In practice, the choice of the number of iterations depends on factors like the size of your dataset, the computational resources available, and how stable you need the estimate to be. Common choices are 5-fold and 10-fold cross-validation.

In summary, while increasing the number of iterations generally leads to a more stable estimate of a model's performance, there are practical considerations to take into account. You'll want to strike a balance between computational resources and the level of confidence you need in your performance estimate.

**4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**

Increasing the number of iterations in cross-validation can help when dealing with a very small dataset. However, it's important to keep in mind some important considerations:

More Stable Estimates: With a very small dataset, there's a higher chance that a single train-test split may not be representative of the overall data distribution. By increasing the number of iterations (e.g., using 10-fold or higher), you're effectively using more of the data for training and validation in each split. This can lead to more stable and reliable performance estimates.

Mitigating Overfitting on Small Datasets: Small datasets are more prone to overfitting. Cross-validation allows you to assess the model's performance on multiple subsets of the data, which can help you gauge whether the model is consistently learning meaningful patterns or if it's just fitting noise in the data.

Practical Limitations: While increasing the number of iterations can be beneficial, there are practical limitations. Extremely small datasets may not provide enough samples to reliably train and evaluate a model, even with cross-validation. In such cases, collecting more data or considering alternative strategies (e.g., transfer learning, data augmentation) may be necessary.

Computational Considerations: With a very small dataset, even a single iteration of cross-validation can be computationally efficient. However, if the dataset is extremely small, the model may struggle to generalize effectively regardless of the number of iterations.

Balancing Train and Validation Set Sizes: As you increase the number of folds, the size of the validation set in each fold decreases. This can be problematic if the validation set becomes too small to provide a meaningful evaluation.

In summary, increasing the number of iterations in cross-validation can be a useful strategy for dealing with very small datasets. However, there are practical limits, and other approaches like data augmentation or transfer learning may also be considered in cases where data scarcity is a significant challenge.
